# Hybrid Search

combine vector, keyword

What does it do:
- it run search on multiple retrievers parallel
- it then use Reciprocal Rank Fusion to sort the document

In [16]:
import os
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]="ls__bccd1a22537542bab44282ee1316a124"
os.environ["LANGCHAIN_PROJECT"]="LEARN"

K = 2 #seach K documents

## BM25 retriever

In [4]:
from __future__ import annotations

from typing import Any, Dict, Iterable, List

from langchain_core.documents import Document
import uuid
from elasticsearch import Elasticsearch

from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun

class MyElasticSearchBM25Retriever(BaseRetriever):
    """`Elasticsearch` retriever that uses `BM25` extended version.
        @param client ElasticSearchClient
        @param index_name an index
    """

    k: int = 4
    """k size search"""

    client: Elasticsearch
    """Elasticsearch client."""

    index_name: str
    """Name of the index to use in Elasticsearch."""

    def add_texts(
        self,
        texts: Iterable[str],
        metadatas: Dict,
        refresh_indices: bool = True,
    ) -> List[str]:
        """Run more texts through the embeddings and add to the retriever.

        Args:
            texts: Iterable of strings to add to the retriever.
            refresh_indices: bool to refresh ElasticSearch indices

        Returns:
            List of ids from adding the texts into the retriever.
        """
        try:
            from elasticsearch.helpers import bulk
        except ImportError:
            raise ValueError(
                "Could not import elasticsearch python package. "
                "Please install it with `pip install elasticsearch`."
            )
        requests = []
        ids = []
        for i, text in enumerate(texts):
            metadata = metadatas[i] if metadatas else {}
            _id = str(uuid.uuid4())
            request = {
                "_op_type": "index",
                "_index": self.index_name,
                "content": text,
                "metadata": metadata,
                "_id": _id,
            }
            ids.append(_id)
            requests.append(request)
        bulk(self.client, requests)

        if refresh_indices:
            self.client.indices.refresh(index=self.index_name)
        return ids
    
    def add_documents(self, documents: List[Document], **kwargs: Any) -> List[str]:
        """Run more documents through the embeddings and add to the vectorstore.

        Args:
            documents (List[Document]: Documents to add to the vectorstore.

        Returns:
            List[str]: List of IDs of the added texts.
        """
        # TODO: Handle the case where the user doesn't provide ids on the Collection
        texts = [doc.page_content for doc in documents]
        metadatas = [doc.metadata for doc in documents]
        return self.add_texts(texts, metadatas, **kwargs)

    def get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        query_dict = {"query": {"match": {"content": query}}, "size": self.k}
        response = self.client.search(index=self.index_name, body=query_dict)

        def default_doc_builder(hit: Dict) -> Document:
            return Document(
                page_content=hit["_source"].get(self.query_field, ""),
                metadata=hit["_source"]["metadata"],
            )
        
        doc_builder = default_doc_builder

        result_docs = []
        for r in response["hits"]["hits"]:
            d = Document(page_content=r["_source"]["content"])
            d.metadata.update(r["_source"]["metadata"])
            result_docs.append(d)
        return result_docs


In [17]:
es = Elasticsearch("http://localhost:9200") # check https://elasticsearch-py.readthedocs.io/en/v8.12.1/

b_index = "text-split-major"

bm25 = MyElasticSearchBM25Retriever(client=es, index_name=b_index)
bm25.k = K

## Vector retriever

In [18]:
from langchain_community.vectorstores.elasticsearch import ElasticsearchStore

embedding = provider.get_gemini_embeddings() # replace with yours

v_index = "labse-major"

elastic_vector_search = ElasticsearchStore(
    es_connection=es,
    index_name=v_index,
    embedding=embedding,
)

vector_retriever = elastic_vector_search.as_retriever(search_kwargs={"k": K})


## Hybrid search

In [19]:
from langchain.retrievers import EnsembleRetriever

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25, vector_retriever],
    weights=[0.5, 0.5])


In [21]:
q = "phần mềm"
docs = ensemble_retriever.invoke(input=q)
docs

[Document(page_content='Công nghệ sinh học là ngành ứng dụng các kiến thức trong lĩnh vực sinh học nhằm nghiên cứu và sản xuất ra các sản phẩm sinh học thiết yếu, sản phẩm có giá trị cao như vaccine, kháng sinh, xăng sinh học; cải tiến chất lượng giống cây trồng, ứng dụng vi sinh để xử lý nước thải công nghiệp, ứng dụng các hợp chất tự nhiên để điều trị bệnh và những ứng dụng khác phục vụ cho đời sống con người.\n\nVới cách tiếp cận và xây dựng chương trình đào tạo hiện đại, tiệm cận với các nền giáo dục tiên tiến và sát với yêu cầu về nguồn nhân lực của thị trường lao động, sinh viên sau khi tốt nghiệp ngành Công nghệ sinh học có thể đảm nhiệm các vị trí như: Phụ trách nghiên cứu và phát triển sản phẩm mới; phụ trách quản lý hệ thống và kiểm định dây chuyền sản xuất của nhà máy; phụ trách theo dõi và kiểm soát vi sinh sản phẩm; chẩn đoán bệnh bằng công nghệ di truyền, liệu pháp gene, công nghệ tế bào gốc; phụ trách theo dõi và phát triển phôi thụ tinh trong ống nghiệm, nghiên cứu v

https://smith.langchain.com/public/44599da9-84dd-49d7-8f23-6da4876eff87/r

https://smith.langchain.com/public/79b56c17-e5b1-4e74-95b5-98b183749477/r